<a href="https://colab.research.google.com/github/amalmon/RETAIL-SALES-PREDICTION/blob/main/AMAL_Rossmann_Sales_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [1]:
import pandas as pd
import numpy as np
from pandas import datetime as dt
from pandas import Series,DataFrame
# data visualization
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.model_selection import train_test_split
# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
rossmann_store_data= pd.read_csv('/content/drive/MyDrive/Almabetter/capstone projects/Capstone project 2 ML regression /Rossmann Stores Data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
store_data=pd.read_csv('/content/drive/MyDrive/Almabetter/capstone projects/Capstone project 2 ML regression /store.csv')

In [68]:
rossmann_df=rossmann_store_data.copy()

In [96]:
store_df=store_data.copy()

In [69]:
rossmann_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [ ]:
store_df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [10]:
rossmann_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [11]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1112 non-null   float64
 4   CompetitionOpenSinceMonth  761 non-null    float64
 5   CompetitionOpenSinceYear   761 non-null    float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            571 non-null    float64
 8   Promo2SinceYear            571 non-null    float64
 9   PromoInterval              571 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [ ]:
rossmann_df.describe(include='all')

In [ ]:
store_df.describe(include='all')

**for duplicate row**

In [ ]:
len(rossmann_df[rossmann_df.duplicated()])

0

In [ ]:
len(store_df[store_df.duplicated()])

0

In [12]:
rossmann_df.DayOfWeek.value_counts()

5    145845
4    145845
3    145665
2    145664
1    144730
7    144730
6    144730
Name: DayOfWeek, dtype: int64

In [13]:
rossmann_df.Open.value_counts()

1    844392
0    172817
Name: Open, dtype: int64

**For null values**

In [ ]:
rossmann_df.isnull().any()

In [ ]:
store_df.isnull().any()

In [16]:
store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [ ]:
store_df.loc[store_df['CompetitionOpenSinceMonth'].isnull()]

**. Here we can understand that null values of competition since month & year is due to missing data why because they have fare values under competition distance.**

**. so, It doesn't mean that there is no competition**

In [ ]:
store_df.loc[store_df['CompetitionDistance'].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
290,291,d,a,NaN,NaN,NaN,0,NaN,NaN,NaN
621,622,a,c,NaN,NaN,NaN,0,NaN,NaN,NaN
878,879,d,a,NaN,NaN,NaN,1,5.0,2013.0,"Feb,May,Aug,Nov"


**. Here we can assume that these 3 stores have no competitors**

So, we can replace NaN values of copetition distance column with maximum value of competition distance available in data
And also we can replace NaN values of column competition since month and year with mode values of the respective columns

In [ ]:
store_df['CompetitionDistance'].max()

75860.0

In [ ]:
store_df['CompetitionDistance'].min()

20.0

In [ ]:
store_df['CompetitionDistance'].mean()

5594.466367713005

In [ ]:
store_df['CompetitionDistance'].median()

2330.0

*here may be no competetors at all for the shopes with null values of competition distance bt based on that when we applying maximum values there is a increase in outliers in column value. so, we choose mean.
and also it will not make significant changes in dataset

In [97]:
# null vales of column CompetitionDistance replaced by mean value
store_df['CompetitionDistance'].fillna(store_df['CompetitionDistance'].mean(), inplace = True)

In [ ]:
#checking

In [ ]:
store_df.loc[290]


In [ ]:
# understanding values of CompetitionOpenSinceMonth

store_df.CompetitionOpenSinceMonth.value_counts()

In [ ]:
#taking mode value of CompetitionOpenSinceMonth

store_df.CompetitionOpenSinceMonth.mode()[0]

9.0

In [98]:
#replacing null values of CompetitionOpenSinceMonth column with its mode value

store_df['CompetitionOpenSinceMonth'].fillna(store_df.CompetitionOpenSinceMonth.mode()[0], inplace = True)

In [ ]:
#checking the changes happen to dataset

store_df.info()

In [99]:
#replacing null values of CompetitionOpenSinceYear with mode value

store_df['CompetitionOpenSinceYear'].fillna(store_df.CompetitionOpenSinceYear.mode()[0], inplace = True)

In [ ]:
#checking the changes happen

store_df.info()

In [53]:
#understanding promo2 column values and its counts

store_df.Promo2.value_counts()

1    571
0    544
Name: Promo2, dtype: int64

In [ ]:
#for mode value of Promo2SinceWeek
store_df.Promo2SinceWeek.mode()

0    14.0
dtype: float64

In [ ]:
#understanding Promo2SinceWeek column values and its counts
store_df.Promo2SinceWeek.value_counts()

In [ ]:
store_df.loc[store_df['Promo2SinceWeek'].isnull()]

**.Here we can conform that the stores which do not participate in promo2 they do not have 
any values in promo2 since yr & month and also promo2 interval**

**. so we can replace the NaN values of promo2 since [yr& month] and promo 2 interval with value 'zero'**

In [ ]:
''' imparting zero value to the null values
in columns promo2 since [yr& month] and promo 2 interval in the rows which contain zero value in promo2 column'''

In [ ]:
store_df.shape[0]

In [ ]:
#putting zero in promosince week where promo 2 is zero

'''for i in range(0,store_df.shape[0]):
  if store_df.iloc[i,6]==0:
    store_df.iloc[i,7]=0'''

* null values of  promo2 since [yr& month] and promo 2 interval can be also directly make it as 'zero' 

In [100]:
#replacing null values of column Promo2SinceYear & PromoInterval with zero
store_df['Promo2SinceWeek'].fillna(0,inplace=True)
store_df['Promo2SinceYear'].fillna(0,inplace=True)
store_df['PromoInterval'].fillna(0,inplace=True)


In [84]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Store                      1115 non-null   int64  
 1   StoreType                  1115 non-null   object 
 2   Assortment                 1115 non-null   object 
 3   CompetitionDistance        1115 non-null   float64
 4   CompetitionOpenSinceMonth  1115 non-null   float64
 5   CompetitionOpenSinceYear   1115 non-null   float64
 6   Promo2                     1115 non-null   int64  
 7   Promo2SinceWeek            1115 non-null   float64
 8   Promo2SinceYear            1115 non-null   float64
 9   PromoInterval              1115 non-null   object 
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


In [101]:
store_df['CompetitionOpenSinceYear']=store_df['CompetitionOpenSinceYear'].astype(int)
store_df['CompetitionOpenSinceMonth']=store_df['CompetitionOpenSinceMonth'].astype(int)

In [ ]:
store_df.CompetitionOpenSinceMonth.value_counts()

In [102]:
#making new column by combining competition since month and year (date assumed to be 15)
store_df['competition_since'] =pd.to_datetime(store_df['CompetitionOpenSinceYear'].astype(str)+'-'+store_df['CompetitionOpenSinceMonth'].astype(str))

In [103]:
del store_df['CompetitionOpenSinceMonth']

In [104]:
del store_df['CompetitionOpenSinceYear']

In [110]:
store_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Store                1115 non-null   int64         
 1   StoreType            1115 non-null   object        
 2   Assortment           1115 non-null   object        
 3   CompetitionDistance  1115 non-null   float64       
 4   Promo2               1115 non-null   int64         
 5   Promo2SinceWeek      1115 non-null   float64       
 6   Promo2SinceYear      1115 non-null   float64       
 7   PromoInterval        1115 non-null   object        
 8   competition_since    1115 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 78.5+ KB


In [111]:
store_df

,Store,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition_since
0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
1,2,a,a,570.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",2007-11-01
2,3,a,a,14130.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",2006-12-01
3,4,c,c,620.0,0,0.0,0.0,0,2009-09-01
4,5,a,a,29910.0,0,0.0,0.0,0,2015-04-01
...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct",2014-06-01
1111,1112,c,c,1880.0,0,0.0,0.0,0,2006-04-01
1112,1113,a,c,9260.0,0,0.0,0.0,0,2013-09-01
1113,1114,a,c,870.0,0,0.0,0.0,0,2013-09-01


In [121]:
# merging 2 datasets on store number

final_df = rossmann_df.merge(store_df,how='inner',on = ['Store'])

In [130]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 17 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   Store                1017209 non-null  int64         
 1   DayOfWeek            1017209 non-null  int64         
 2   Date                 1017209 non-null  object        
 3   Sales                1017209 non-null  int64         
 4   Customers            1017209 non-null  int64         
 5   Open                 1017209 non-null  int64         
 6   Promo                1017209 non-null  int64         
 7   StateHoliday         1017209 non-null  object        
 8   SchoolHoliday        1017209 non-null  int64         
 9   StoreType            1017209 non-null  object        
 10  Assortment           1017209 non-null  object        
 11  CompetitionDistance  1017209 non-null  float64       
 12  Promo2               1017209 non-null  int64         
 1

In [131]:
#converting object typ of column 'date' to date time for subtract with 'competition since'
final_df['Date']=pd.to_datetime(final_df['Date'])

In [132]:
final_df['competition_since_before'] =(final_df['Date']-final_df['competition_since'])

In [153]:
del final_df['competition_since']

In [ ]:
final_df['competition_since_before']=final_df['competition_since_before']/np.timedelta64(1,'M')

In [154]:
final_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition_since_before
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,82.925727
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,82.892873
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,82.860018
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,82.827163
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,82.794308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",-7.852317
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",-7.885172
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",-7.918027
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",-7.950882


In [155]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 17 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   Store                     1017209 non-null  int64         
 1   DayOfWeek                 1017209 non-null  int64         
 2   Date                      1017209 non-null  datetime64[ns]
 3   Sales                     1017209 non-null  int64         
 4   Customers                 1017209 non-null  int64         
 5   Open                      1017209 non-null  int64         
 6   Promo                     1017209 non-null  int64         
 7   StateHoliday              1017209 non-null  object        
 8   SchoolHoliday             1017209 non-null  int64         
 9   StoreType                 1017209 non-null  object        
 10  Assortment                1017209 non-null  object        
 11  CompetitionDistance       1017209 non-null  float6

In [146]:
final_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition_since,competition_since_before
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.925727
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.892873
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.860018
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.827163
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.794308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.852317
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.885172
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.918027
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.950882


In [161]:
#all negative values of 'competition_since_before' convert to 0 value, why because it means that the competition not started
final_df.loc[final_df['competition_since_before']<0,'competition_since_before']=0

In [165]:
#converting objct typ to intiger
final_df['competition_since_before']=final_df['competition_since_before'].astype(int)

In [167]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 17 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   Store                     1017209 non-null  int64         
 1   DayOfWeek                 1017209 non-null  int64         
 2   Date                      1017209 non-null  datetime64[ns]
 3   Sales                     1017209 non-null  int64         
 4   Customers                 1017209 non-null  int64         
 5   Open                      1017209 non-null  int64         
 6   Promo                     1017209 non-null  int64         
 7   StateHoliday              1017209 non-null  object        
 8   SchoolHoliday             1017209 non-null  int64         
 9   StoreType                 1017209 non-null  object        
 10  Assortment                1017209 non-null  object        
 11  CompetitionDistance       1017209 non-null  float6

In [196]:
final_df['Promo2SinceWeek']=final_df['Promo2SinceWeek'].astype(int)

final_df['Promo2SinceYear']=final_df['Promo2SinceYear'].astype(int)

In [179]:
final_df['promo2month']=final_df['Promo2SinceWeek']/4

In [183]:
final_df['promo2month']=final_df['promo2month'].astype(int)

In [ ]:
final_df['Promo2_since_date']=

In [186]:
final_df['promo2month'].value_counts()

0     540449
3     102810
10     62598
2      51280
9      46754
7      40918
1      36760
5      33036
11     31910
4      27318
8      22814
6      10568
12      9994
Name: promo2month, dtype: int64

In [197]:
final_df['promo2_since_date']=final_df['Promo2SinceYear'].astype(str)+'-'+final_df['promo2month'].astype(str)

In [194]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 19 columns):
 #   Column                    Non-Null Count    Dtype         
---  ------                    --------------    -----         
 0   Store                     1017209 non-null  int64         
 1   DayOfWeek                 1017209 non-null  int64         
 2   Date                      1017209 non-null  datetime64[ns]
 3   Sales                     1017209 non-null  int64         
 4   Customers                 1017209 non-null  int64         
 5   Open                      1017209 non-null  int64         
 6   Promo                     1017209 non-null  int64         
 7   StateHoliday              1017209 non-null  object        
 8   SchoolHoliday             1017209 non-null  int64         
 9   StoreType                 1017209 non-null  object        
 10  Assortment                1017209 non-null  object        
 11  CompetitionDistance       1017209 non-null  float6

In [198]:
final_df['promo2_since_date']=pd.to_datetime(final_df['promo2_since_date'])

ParserError: ignored

In [ ]:
final_df['Promo2_since'] = ( 12 * (final_df['Date'] - final_df['Promo2SinceYear'])) + df2['Month'] - ((df2['Promo2SinceWeek']/4))

***onehot encoding***

In [61]:
#making dummies for chatagorical columns
store_df=pd.get_dummies(store_df,columns=['StoreType','Assortment','PromoInterval'])

In [ ]:
store_df

In [ ]:
rossmann_df.SchoolHoliday.value_counts()

0    835488
1    181721
Name: SchoolHoliday, dtype: int64

In [ ]:
rossmann_df.StateHoliday.value_counts()

0    855087
0    131072
a     20260
b      6690
c      4100
Name: StateHoliday, dtype: int64

In [66]:
rossmann_df=pd.get_dummies(rossmann_df,columns=['StateHoliday'])

In [ ]:
rossmann_df

In [ ]:
rossmann_store_data.value_counts

In [120]:
rossmann_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [ ]:
store_df.info()

In [ ]:
rossmann_df.info()

In [ ]:
store_df

In [ ]:
final_df.drop('PromoInterval_0',axis=1,inplace=True)

In [ ]:
rossmann_df["Date"]=pd.to_datetime(rossmann_df["Date"])
#final_df["Year"]=final_df["Date"].dt.year
#final_df["Month"]=final_df["Date"].dt.month
#final_df["Day"]=final_df["Date"].dt.day
#final_df["Week"]=final_df["Date"].dt.week%4

In [ ]:
'''#creating bar plot of promo intervel with averg. sales
ax=final_df.groupby(['PromoInterval'])['Sales'].sum().plot.bar(figsize=(15,5),fontsize=15)

#giving tittle
ax.set_title('average sales versus promotion intervals',fontsize=15)'''




In [ ]:
rossmann_df.info()

In [ ]:
final_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition since
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01


In [ ]:
final_df.info()

In [ ]:
final_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition since,competition since before
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,2524 days
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,2523 days
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,2522 days
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,2521 days
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,2520 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-239 days
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-240 days
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-241 days
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-242 days


In [ ]:
final_df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,competition since,competition since before
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.925727
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.892873
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.860018
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.827163
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,0,0.0,0.0,0,2008-09-01,82.794308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1115,6,2013-01-05,4771,339,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.852317
1017205,1115,5,2013-01-04,4540,326,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.885172
1017206,1115,4,2013-01-03,4297,300,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.918027
1017207,1115,3,2013-01-02,3697,305,1,0,0,1,d,c,5350.0,1,22.0,2012.0,"Mar,Jun,Sept,Dec",2013-09-01,-7.950882


In [ ]:
#taking difference of date and competition since date and making a single column with competition starts before how many days


Here we have promo since week and year.
we have to convert it into a date format 

Here week represents number of week in a calender year so, we have to convert it into month and day

In [ ]:
plt.scatterplot